In [38]:
import scipy.io.wavfile as wav
import numpy as np
import shutil
import os

In [35]:
input_file = 'test-input.wav'
output_dir = 'filter-output'
seg_secs = 1
fold_ratio = 0.1

In [29]:
sample_rate, samples = wav.read(input_file)
seg_samples = round(seg_secs * sample_rate)
segs = [samples[seg_samples*i:seg_samples*(i+1)] for i in range(len(samples) // seg_samples)]
assert len(segs) > 0 and all(x.shape == segs[0].shape for x in segs)

In [43]:
background_freq = np.fft.fft(segs[0])
energy_thresh = 0
keeps = []
for i, seg in enumerate(segs):
    freq = np.fft.fft(seg)
    energy = np.sum((freq - background_freq)**2)
    if energy > energy_thresh:
        keeps.append((i, seg))
    background_freq = (1 - fold_ratio) * background_freq + fold_ratio * freq
    energy_thresh = (1 - fold_ratio) * energy + fold_ratio * energy_thresh

print(f'before: {len(segs)}')
print(f'after:  {len(keeps)}')

before: 1215
after:  590


In [44]:
shutil.rmtree(output_dir, ignore_errors = True)
os.mkdir(output_dir)
for i, keep in keeps:
    wav.write(f'{output_dir}/keep-{i}.wav', sample_rate, keep)